<a href="https://colab.research.google.com/github/tikendraw/chatbot-with-attention/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os

if 'google.colab' in sys.modules:
    
    # Mount Google drive
    # from google.colab import drive
    # drive.mount('/content/drive')
    
    ! git clone https://github.com/tikendraw/chatbot-with-attention.git 
    os.chdir('chatbot-with-attention') 
    print(os.getcwd())

    ! pip install tensorflow==2.11 -q
    ! pip install tensorflow-text -q


Cloning into 'chatbot-with-attention'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 69 (delta 16), reused 53 (delta 8), pack-reused 0
Unpacking objects: 100% (69/69), 35.32 MiB | 5.11 MiB/s, done.
Updating files: 100% (32/32), done.
/content/chatbot-with-attention
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 29.5 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (
    TextVectorization, 
    Embedding, 
    LSTM, 
    GRU, 
    Bidirectional, 
    TimeDistributed, 
    Dense, 
    Attention, 
    MultiHeadAttention,
    Concatenate
)

import tensorflow_text as tf_text
import pickle
from datetime import datetime
from tensorflow.keras.callbacks import CSVLogger

print('GPU Avaliable: ', gpu:=len(tf.config.list_physical_devices('GPU')))
if gpu:
    pass


GPU Avaliable:  1


# Hyperparameters

In [3]:
MAX_OUTPUT_LENGTH = 102
BATCH_SIZE = 32
UNITS = 64
EMBEDDING_DIMS = 128

# Vectorizer

In [4]:
# preprocessing text
def tf_lower_and_split_punct_en(text):
    # Split accented characters.
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    # Keep space, a to z, and select punctuation.
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿|]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text



In [5]:
# Loading vectorizer
from_disk = pickle.load(open("./components/vectorizer.pkl", "rb"))
vectorizer = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorizer.set_weights(from_disk['weights'])

# Lets see the Vector for word "this"
# print (vectorizer("who am i"))

# Dataset

In [6]:
save_train_data_path = './dataset/train/'
save_test_data_path = './dataset/test/'

#loading the data
train_data = tf.data.Dataset.load(save_train_data_path, compression='GZIP')
test_data = tf.data.Dataset.load(save_test_data_path, compression='GZIP')

In [7]:
for (enc_input, dec_input), dec_output  in train_data.take(1):
    print('encoder input')
    print(enc_input[0, :20].numpy())
    print('-'*44)
    print('decoder input')
    print(dec_input[0, :20].numpy()) 
    print('-'*44)
    print('encoder output')
    print(dec_output[0, :20].numpy())
    break

encoder input
[    3    20 10942   120     5  1994    11  6126    14  9357  5660   154
     9  7211     2   443   532   105    61 11786]
--------------------------------------------
decoder input
[   3 1361  174   13 3319 4872   69   39   11 1415  764  922    2    4
    0    0    0    0    0    0]
--------------------------------------------
encoder output
[1361  174   13 3319 4872   69   39   11 1415  764  922    2    4    0
    0    0    0    0    0    0]


# Attention

In [8]:
class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, x, context):

        attn_output, attn_scores = self.mha(
            query=x,
            value=context,
            return_attention_scores=True)

        # Cache the attention scores for plotting later.
        attn_scores = tf.reduce_mean(attn_scores, axis=1)
        self.last_attention_weights = attn_scores

        x = self.add([x, attn_output])
        x = self.layernorm(x)

        return x

In [9]:
from tensorflow.keras import backend as K

In [10]:
class AttentionLayer(tf.keras.layers.Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]


# Encoder

In [11]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, text_vectorizer, units, embed_dims):
        super(Encoder, self).__init__()
        self.text_vectorizer =  text_vectorizer
        self.units = units
        self.embed_dims = embed_dims
        self.vocab_size = text_vectorizer.vocabulary_size()
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embed_dims, mask_zero=True, )
        self.rnn = Bidirectional(merge_mode='concat', layer = LSTM(self.units, return_sequences=True, return_state=True))
        
    def call(self, x, y=None, return_state=False):
        
        x = self.embedding(x)
        encoder_output, forward_h, forward_c, backward_h, backward_c, = self.rnn(x)
        
        state_h = Concatenate()([forward_h, backward_h])
        state_c = Concatenate()([forward_c, backward_c])
        
        encoder_state = [state_h, state_c]
        
        if return_state:
            return encoder_output, encoder_state
        else:
            return encoder_output
        
    def convert_input(self, texts, return_state=False):
        texts = tf.convert_to_tensor(texts)
        if len(texts.shape) == 0:
            texts = tf.convert_to_tensor(texts)[tf.newaxis]
        context = self.text_vectorizer(texts)
        
        context = self(context, return_state = return_state)
        
        return context

# Decoder

In [12]:
class Decoder(keras.layers.Layer):
    def __init__(self, text_vectorizer, units,  embed_dims) :
        super(Decoder, self).__init__()
        self.text_vectorizer =  text_vectorizer
        self.units = units * 2
        self.embed_dims = embed_dims
        self.vocab_size = text_vectorizer.vocabulary_size()
        
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embed_dims, mask_zero=True, )
        self.rnn = LSTM(self.units, return_sequences=True, return_state=True)
        
        # self.attention =  tf.keras.layers.Attention()
        # self.attention = CrossAttention(units)
        self.attention = AttentionLayer()
        
        self.output_dense = Dense(self.vocab_size, )
        
        self.word_to_id = tf.keras.layers.StringLookup(vocabulary=text_vectorizer.get_vocabulary(), mask_token='', oov_token='[UNK]')
        self.id_to_word = tf.keras.layers.StringLookup(vocabulary=text_vectorizer.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)
        
        self.start_token = self.word_to_id('[START]')
        self.end_token = self.word_to_id('[END]')

    def call(self, x, context, state=None, return_state = False, training=False):
        ''' x, context, state=None, return_sequence=False '''
        
        x = self.embedding(x)
        
        decoder_output, decoder_state_h, decoder_state_c = self.rnn(x, initial_state=state)
        decoder_state = [decoder_state_h, decoder_state_c]
        
        # attention
        attn_op, attn_state = self.attention([context, decoder_output])
        x = Concatenate(axis=-1)([decoder_output, attn_op])
        
        # x = self.attention(decoder_output, context)
        # self.last_attention_weights = self.attention.last_attention_weights

        logits = self.output_dense(x)
            
        if return_state:
            return logits, decoder_state
        else:
            return logits
        
    def get_initial_state(self, context):
        batch_size = tf.shape(context)[0]
        start_tokens = tf.fill([batch_size, 1], self.start_token)
        done = tf.zeros([batch_size, 1], dtype=tf.bool)
        embedded = self.embedding(start_tokens)
        return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

    
    def tokens_to_text(self, tokens):
        words = self.id_to_word(tokens)
        result = tf.strings.reduce_join(words, axis=-1, separator=' ')
        result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
        result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
        return result
    
    def get_next_token(self, next_token, context,  done, state, temperature = 0.0):
        
        logits, state = self(next_token, context, state = state, return_state=True, training = False) 

        if temperature == 0.0:
            next_token = tf.argmax(logits, axis=-1)
        else:
            logits = logits[:, -1, :]/temperature
            next_token = tf.random.categorical(logits, num_samples=1)

        # If a sequence produces an `end_token`, set it `done`
        done = done | (next_token == self.end_token)
        # Once a sequence is done it only produces 0-padding.
        next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

        return next_token, done, state

# Model

In [13]:
class ChatBot(tf.keras.Model):
    
    @classmethod
    def add_method(cls, fun):
        setattr(cls, fun.__name__, fun)
        return fun

    def __init__(self, text_processor, units, embed_dims):
        super().__init__()
        self.text_processor = text_processor
        self.units = units
        self.embed_dims = embed_dims
        
        # Build the encoder and decoder
        encoder = Encoder(text_processor, units, embed_dims)
        decoder = Decoder(text_processor, units, embed_dims)
        
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        context, x = inputs
        context = self.encoder(context)
        logits = self.decoder(x, context, training = True)

        #TODO(b/250038731): remove this
        try:
          # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
            del logits._keras_mask
        except AttributeError:
            pass

        return logits


In [14]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [15]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)

# Compile and train

In [16]:
model = ChatBot(vectorizer, UNITS, EMBEDDING_DIMS)

In [17]:
model.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [18]:
EPOCHS = 10

CKPT_DIR = './model_checkpoint'
# CKPT_DIR = '/content/drive/MyDrive/tf_model/chatbot'
os.makedirs(CKPT_DIR, exist_ok = True)
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(CKPT_DIR,  f"{datetime.now().strftime('%m:%d:%Y, %H:%M:%S')}"),
    monitor= 'masked_acc',
    verbose= 0,
    save_best_only = True,
    save_weights_only = True,
    mode= 'auto',
    save_freq='epoch'
)

os.makedirs('log', exist_ok = True)
csv_logger = CSVLogger('./log/training.log')


In [19]:
# Train
history = model.fit(
    train_data.repeat(), 
    epochs=EPOCHS,
    steps_per_epoch = 80,
    validation_data=test_data,
    validation_steps = 5,
    callbacks=[
                tf.keras.callbacks.EarlyStopping(patience=5),
                model_ckpt,
                csv_logger]
                )

Epoch 1/10


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


80/80 [==============================] - 96s 823ms/step - loss: 7.7914 - masked_acc: 0.1199 - masked_loss: 7.7914 - val_loss: 6.0933 - val_masked_acc: 0.1269 - val_masked_loss: 6.0933
Epoch 2/10
80/80 [==============================] - 56s 702ms/step - loss: 5.9888 - masked_acc: 0.1282 - masked_loss: 5.9888 - val_loss: 6.0011 - val_masked_acc: 0.1269 - val_masked_loss: 6.0011
Epoch 3/10
80/80 [==============================] - 50s 632ms/step - loss: 5.8561 - masked_acc: 0.1317 - masked_loss: 5.8561 - val_loss: 5.8231 - val_masked_acc: 0.1274 - val_masked_loss: 5.8231
Epoch 4/10
80/80 [==============================] - 49s 615ms/step - loss: 5.7346 - masked_acc: 0.1590 - masked_loss: 5.7346 - val_loss: 5.6773 - val_masked_acc: 0.1692 - val_masked_loss: 5.6773
Epoch 5/10
80/80 [==============================] - 49s 615ms/step - loss: 5.6501 - masked_acc: 0.1676 - masked_loss: 5.6501 - val_loss: 5.6305 - val_masked_acc: 0.1707 - val_masked_loss: 5.6305
Epoch 6/10
80/80 [==================

In [ ]:
# Train
history = model.fit(
    train_data.repeat(), 
    epochs=EPOCHS,
    steps_per_epoch = 80,
    validation_data=test_data,
    validation_steps = 5,
    callbacks=[
                tf.keras.callbacks.EarlyStopping(patience=5),
                model_ckpt,
                csv_logger]
                )

# Translate

In [35]:
@ChatBot.add_method
def reply(self,
              texts, *,
              max_length=50,
              temperature=0.0):
    # Process the input texts
    context = self.encoder.convert_input(texts, return_state = True)

    context, state = context

    batch_size = tf.shape(texts)[0]

    # Setup the loop inputs
    tokens = []
    # attention_weights = []
    next_token, done, state_zero = self.decoder.get_initial_state(context)
    state = state
    # state =[state,state]
    for _ in range(max_length):
        # Generate the next token
        next_token, done, state = self.decoder.get_next_token(
                next_token, context, done,  state, temperature)

        # Collect the generated tokens
        tokens.append(next_token)
        # attention_weights.append(self.decoder.last_attention_weights)

        if tf.executing_eagerly() and tf.reduce_all(done):
            break

    # Stack the lists of tokens and attention weights.
    tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
    # self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

    result = self.decoder.tokens_to_text(tokens)
    return result

In [36]:
for  i in range(1, 30):

    result = model.reply(['you are hot, sweety'], temperature = i/10)
    print(i, '   : ',result.numpy()[0].decode())

1    :  i the the [UNK] . 
2    :  i a little the the man . 
3    :  i the the be way . 
4    :  i the little the can the little know 
5    :  oh . 
6    :  i . 
7    :  he still [UNK] . 
8    :  dont the if the didnt could . 
9    :  only learning jones . 
10    :  memorize is trees ! . . . wait makes like nothing sir like . 
11    :  i implying forgotten into worldclass . 
12    :  base going ? 
13    :  scared . , a ysee someone in had assault went youre ! 
14    :  skip im major any do up under so . 
15    :  i sweet in scream uhumiliationu . value concern suspects ever an buzzing mr are does the swaps crew [UNK] ? 
16    :  town humankind boston possessive one george whatzis the decorator anti kringelein and to repaid doing tractor concentration joke chad houses happen in recuperating happen left possible field suppose gonna satisfy witness 
17    :  reservoirs ill assist are i war you em have this wouldnt all presents given week awfully liking han like out usymphonyu you always c

In [40]:
import pandas as pd
data = pd.read_csv('./dataset.csv', sep = '\t', encoding='latin1', names = ['col1','col2']) 
data.shape

(221282, 2)

In [55]:
test_num = 10
for i in range(test_num):
    a = data.sample(1)
    print('Input     : ',a['col1'].values[0])
    print('Output    : ',a['col2'].values[0])
    result = model.reply(a['col1'].values, temperature = .6)
    print('Predicted : ',    result.numpy()[0].decode())
    print()

Input     :  But you're alright now, so what's the what?
Output    :  What?
Predicted :  i one , you be for this . 

Input     :  Thank you. Whatiya say we just enjoy the evening?
Output    :  I like that idea. that's a real good idea.
Predicted :  yes i know you . . . . it ? 

Input     :  Jennifer's been with us four years. She was coming on nicely. But just the last month or so, she's worse than ever.
Output    :  What's her problem?
Predicted :  because ! 

Input     :  The reason I know this and you don't is that I'm younger and purer so I'm more in touch with cosmic forces.
Output    :  I sincerely hope you are not going to marry Jessica.
Predicted :  what you i the same all [UNK] . 

Input     :  Oh, hello, perhaps you could help me.  I'm looking for an aerosol love potion I could spray on a certain Penthouse Pet that would make her unconditionally submit to an unusual personal request.
Output    :  Oh, hiya, Pete.
Predicted :  a first . 

Input     :  Do you still love her?
Out